In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="/data/yinxiaoln/datasets",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="/data/yinxiaoln/datasets",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork()

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [4]:
loss_fn = nn.CrossEntropyLoss()

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [21]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [22]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
print(model.parameters().__next__().device)
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

cpu
Epoch 1
-------------------------------
loss: 0.580812  [   64/60000]
loss: 0.687507  [ 6464/60000]
loss: 0.464984  [12864/60000]
loss: 0.721269  [19264/60000]
loss: 0.632680  [25664/60000]
loss: 0.609588  [32064/60000]
loss: 0.675917  [38464/60000]
loss: 0.702763  [44864/60000]
loss: 0.676439  [51264/60000]
loss: 0.634037  [57664/60000]
Test Error: 
 Accuracy: 77.9%, Avg loss: 0.635511 

Epoch 2
-------------------------------
loss: 0.564331  [   64/60000]
loss: 0.671996  [ 6464/60000]
loss: 0.452369  [12864/60000]
loss: 0.710472  [19264/60000]
loss: 0.624255  [25664/60000]
loss: 0.601328  [32064/60000]
loss: 0.661242  [38464/60000]
loss: 0.695226  [44864/60000]
loss: 0.668234  [51264/60000]
loss: 0.622666  [57664/60000]
Test Error: 
 Accuracy: 78.3%, Avg loss: 0.624279 

Epoch 3
-------------------------------
loss: 0.549313  [   64/60000]
loss: 0.657686  [ 6464/60000]
loss: 0.440832  [12864/60000]
loss: 0.700485  [19264/60000]
loss: 0.616600  [25664/60000]
loss: 0.593902  [32064